In [1]:
import sys
sys.path.append("../")

In [2]:
from mir.common.parser import *

In [3]:
from mir.common.repertoire import Repertoire
from mir.common.repertoire_dataset import RepertoireDataset
from mir.basic.segment_usage import *
from mir.basic.sampling import RepertoireSampling
import time

In [4]:
from sklearn.manifold import TSNE

In [5]:
metadata = pd.read_csv('assets/samples/fmba_healthy/metadata.csv')
metadata

,sequencingDate,folder,R1,R2,name,COVID_status,projectName,COVID_IgG,COVID_IgM,COVID_PCR,...,HLA-DQB1.2,HLA-DRB1.1,HLA-DRB1.2,id,subsequent_sequencing,HLA-DRB4.1,HLA-DRB3.1,HLA-DRB5.1,gene,run
0,2020 10 19,2020/10_FMBA_NovaSeq6,770101520808_S102_L002_R1_001.fastq.gz,770101520808_S102_L002_R2_001.fastq.gz,770101520808,healthy,FMBA_DNA,no,no,negative,...,DQB1*06:02,DRB1*11:04,DRB1*15:01,770101520,NaN,NaN,NaN,NaN,NaN,770101520808_S102_L002.clonotypes.TRB.txt
1,2020 10 19,2020/10_FMBA_NovaSeq6,500001210807_S66_L001_R1_001.fastq.gz,500001210807_S66_L001_R2_001.fastq.gz,500001210807,healthy,FMBA_DNA,no,no,negative,...,DQB1*04:02,DRB1*07:01,DRB1*08:01,500001210,NaN,NaN,NaN,NaN,NaN,500001210807_S66_L001.clonotypes.TRB.txt
2,2020 10 19,2020/10_FMBA_NovaSeq6,500002190807_S8_L001_R1_001.fastq.gz,500002190807_S8_L001_R2_001.fastq.gz,500002190807,healthy,FMBA_DNA,no,no,negative,...,DQB1*04:02,DRB1*07:01,DRB1*08:01,500002190,NaN,NaN,NaN,NaN,NaN,500002190807_S8_L001.clonotypes.TRB.txt
3,2020 10 19,2020/10_FMBA_NovaSeq4,780003510808_S92_L001_R1_001.fastq.gz,780003510808_S92_L001_R2_001.fastq.gz,780003510808,healthy,FMBA_DNA,no,no,negative,...,DQB1*03:03,DRB1*07:01,DRB1*07:01,780003510,NaN,NaN,NaN,NaN,NaN,780003510808_S92_L001.clonotypes.TRB.txt
4,2020 10 19,2020/10_FMBA_NovaSeq4,550001610808_S148_L002_R1_001.fastq.gz,550001610808_S148_L002_R2_001.fastq.gz,550001610808,healthy,FMBA_DNA,no,no,negative,...,DQB1*05:01,DRB1*01:01,DRB1*11:04,550001610,NaN,NaN,NaN,NaN,NaN,550001610808_S148_L002.clonotypes.TRB.txt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2020 10 19,2020/10_FMBA_NovaSeq5,270000140808_S174_L002_R1_001.fastq.gz,270000140808_S174_L002_R2_001.fastq.gz,270000140808,healthy,FMBA_DNA,no,no,negative,...,DQB1*05:02,DRB1*16:01,DRB1*16:01,270000140,NaN,NaN,NaN,NaN,NaN,270000140808_S174_L002.clonotypes.TRB.txt
96,2020 10 19,2020/10_FMBA_NovaSeq5,500001400808_S155_L002_R1_001.fastq.gz,500001400808_S155_L002_R2_001.fastq.gz,500001400808,healthy,FMBA_DNA,no,no,negative,...,DQB1*06:01,DRB1*07:01,DRB1*15:02,500001400,NaN,NaN,NaN,NaN,NaN,500001400808_S155_L002.clonotypes.TRB.txt
97,2020 10 19,2020/10_FMBA_NovaSeq6,780004000807_S5_L001_R1_001.fastq.gz,780004000807_S5_L001_R2_001.fastq.gz,780004000807,healthy,FMBA_DNA,no,no,negative,...,DQB1*05:01,DRB1*01:01,DRB1*04:01,780004000,NaN,NaN,NaN,NaN,NaN,780004000807_S5_L001.clonotypes.TRB.txt
98,2020 10 19,2020/10_FMBA_NovaSeq4,550001850808_S122_L002_R1_001.fastq.gz,550001850808_S122_L002_R2_001.fastq.gz,550001850808,healthy,FMBA_DNA,no,no,negative,...,DQB1*05:01,DRB1*01:01,DRB1*04:01,550001850,NaN,NaN,NaN,NaN,NaN,550001850808_S122_L002.clonotypes.TRB.txt


In [6]:
t0 = time.time()
dataset = RepertoireDataset.load(parser=VDJtoolsParser(sep=','), 
                                 metadata=metadata,
                                 threads=28,
                                 paths=['assets/samples/fmba_healthy/' + r['run'] + '.gz'
                                        for _, r in metadata.iterrows()])
print(time.time() - t0)


1444976
131.62847566604614


In [7]:
dataset.metadata

,sequencingDate,folder,R1,R2,name,COVID_status,projectName,COVID_IgG,COVID_IgM,COVID_PCR,...,HLA-DRB1.1,HLA-DRB1.2,id,subsequent_sequencing,HLA-DRB4.1,HLA-DRB3.1,HLA-DRB5.1,gene,run,path
0,2020 10 19,2020/10_FMBA_NovaSeq6,770101520808_S102_L002_R1_001.fastq.gz,770101520808_S102_L002_R2_001.fastq.gz,770101520808,healthy,FMBA_DNA,no,no,negative,...,DRB1*11:04,DRB1*15:01,770101520,NaN,NaN,NaN,NaN,NaN,770101520808_S102_L002.clonotypes.TRB.txt,assets/samples/fmba_healthy/770101520808_S102_...
1,2020 10 19,2020/10_FMBA_NovaSeq6,500001210807_S66_L001_R1_001.fastq.gz,500001210807_S66_L001_R2_001.fastq.gz,500001210807,healthy,FMBA_DNA,no,no,negative,...,DRB1*07:01,DRB1*08:01,500001210,NaN,NaN,NaN,NaN,NaN,500001210807_S66_L001.clonotypes.TRB.txt,assets/samples/fmba_healthy/500001210807_S66_L...
2,2020 10 19,2020/10_FMBA_NovaSeq6,500002190807_S8_L001_R1_001.fastq.gz,500002190807_S8_L001_R2_001.fastq.gz,500002190807,healthy,FMBA_DNA,no,no,negative,...,DRB1*07:01,DRB1*08:01,500002190,NaN,NaN,NaN,NaN,NaN,500002190807_S8_L001.clonotypes.TRB.txt,assets/samples/fmba_healthy/500002190807_S8_L0...
3,2020 10 19,2020/10_FMBA_NovaSeq4,780003510808_S92_L001_R1_001.fastq.gz,780003510808_S92_L001_R2_001.fastq.gz,780003510808,healthy,FMBA_DNA,no,no,negative,...,DRB1*07:01,DRB1*07:01,780003510,NaN,NaN,NaN,NaN,NaN,780003510808_S92_L001.clonotypes.TRB.txt,assets/samples/fmba_healthy/780003510808_S92_L...
4,2020 10 19,2020/10_FMBA_NovaSeq4,550001610808_S148_L002_R1_001.fastq.gz,550001610808_S148_L002_R2_001.fastq.gz,550001610808,healthy,FMBA_DNA,no,no,negative,...,DRB1*01:01,DRB1*11:04,550001610,NaN,NaN,NaN,NaN,NaN,550001610808_S148_L002.clonotypes.TRB.txt,assets/samples/fmba_healthy/550001610808_S148_...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2020 10 19,2020/10_FMBA_NovaSeq5,270000140808_S174_L002_R1_001.fastq.gz,270000140808_S174_L002_R2_001.fastq.gz,270000140808,healthy,FMBA_DNA,no,no,negative,...,DRB1*16:01,DRB1*16:01,270000140,NaN,NaN,NaN,NaN,NaN,270000140808_S174_L002.clonotypes.TRB.txt,assets/samples/fmba_healthy/270000140808_S174_...
96,2020 10 19,2020/10_FMBA_NovaSeq5,500001400808_S155_L002_R1_001.fastq.gz,500001400808_S155_L002_R2_001.fastq.gz,500001400808,healthy,FMBA_DNA,no,no,negative,...,DRB1*07:01,DRB1*15:02,500001400,NaN,NaN,NaN,NaN,NaN,500001400808_S155_L002.clonotypes.TRB.txt,assets/samples/fmba_healthy/500001400808_S155_...
97,2020 10 19,2020/10_FMBA_NovaSeq6,780004000807_S5_L001_R1_001.fastq.gz,780004000807_S5_L001_R2_001.fastq.gz,780004000807,healthy,FMBA_DNA,no,no,negative,...,DRB1*01:01,DRB1*04:01,780004000,NaN,NaN,NaN,NaN,NaN,780004000807_S5_L001.clonotypes.TRB.txt,assets/samples/fmba_healthy/780004000807_S5_L0...
98,2020 10 19,2020/10_FMBA_NovaSeq4,550001850808_S122_L002_R1_001.fastq.gz,550001850808_S122_L002_R2_001.fastq.gz,550001850808,healthy,FMBA_DNA,no,no,negative,...,DRB1*01:01,DRB1*04:01,550001850,NaN,NaN,NaN,NaN,NaN,550001850808_S122_L002.clonotypes.TRB.txt,assets/samples/fmba_healthy/550001850808_S122_...


In [8]:
dataset.evaluate_segment_usage()

,TRBJ2-1*01,TRBJ1-3*01,TRBV11-3*01,TRBV7-3*01,TRBV12-2*01,TRBJ2-6*01,TRBJ1-5*01,TRBV15*01,TRBV27*01,TRBV7-7*01,...,TRBJ1-2*01,TRBJ2-7*01,TRBV5-1*01,TRBV12-3*01,TRBV2*01,TRBV14*01,TRBV6-3*01,TRBV6-9*01,TRBJ2-2P*01,TRBJ2-2*01
0,2716,999,191,393,50,404,2080,114,837,94,...,2693,2195,1381,841,825,298,1,10,0,1558
1,1334,255,62,117,9,160,522,5,286,27,...,670,666,303,307,171,122,0,11,0,586
2,1708,329,122,185,25,214,754,17,364,47,...,1210,1360,515,540,397,147,1,15,0,755
3,543,120,49,69,8,84,269,10,112,19,...,398,379,174,172,114,71,0,8,0,262
4,3454,808,226,539,71,444,1640,33,664,114,...,2523,2181,1166,986,837,250,0,26,1,1620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,4060,908,293,452,69,441,2113,45,1047,119,...,2435,2423,1600,863,1007,374,1,17,0,1902
96,876,154,34,36,13,116,390,10,221,21,...,506,366,332,229,239,88,0,5,0,449
97,1022,453,262,144,21,213,893,21,423,27,...,1645,1752,431,469,232,135,0,5,0,362
98,2102,382,136,330,29,283,943,20,539,52,...,1039,1234,629,459,279,138,1,6,1,927


In [9]:
folder_to_run_mapping = {}
for folder in dataset.metadata[['run', 'folder']].folder.unique():
    folder_to_run_mapping[folder] = set(dataset.metadata[dataset.metadata.folder == folder].run)

In [10]:
norm_usage_table_v = NormalizedSegmentUsageTable.load_from_repertoire_dataset(repertoire_dataset=dataset, gene='TRB', segment_type='V')
norm_usage_table_j = NormalizedSegmentUsageTable.load_from_repertoire_dataset(repertoire_dataset=dataset, gene='TRB', segment_type='J')

z_score_usage_table_v = StandardizedSegmentUsageTable.load_from_repertoire_dataset(repertoire_dataset=dataset, gene='TRB', segment_type='V', group_mapping=folder_to_run_mapping, standardization_method='log_exp')
z_score_usage_table_j = StandardizedSegmentUsageTable.load_from_repertoire_dataset(repertoire_dataset=dataset, gene='TRB', segment_type='J', group_mapping=folder_to_run_mapping, standardization_method='log_exp')


In [11]:
dataset[0].number_of_reads

7823059

In [12]:
norm_usage_table_v

    TRBV11-3*01  TRBV7-3*01  TRBV12-2*01  TRBV15*01  TRBV27*01  TRBV7-7*01  \
0      0.008248    0.016971     0.002159   0.004923   0.036145    0.004059   
1      0.008430    0.015908     0.001224   0.000680   0.038885    0.003671   
2      0.011086    0.016811     0.002272   0.001545   0.033076    0.004271   
3      0.013443    0.018930     0.002195   0.002743   0.030727    0.005213   
4      0.010134    0.024169     0.003184   0.001480   0.029774    0.005112   
..          ...         ...          ...        ...        ...         ...   
95     0.011301    0.017433     0.002661   0.001736   0.040381    0.004590   
96     0.006522    0.006906     0.002494   0.001918   0.042394    0.004028   
97     0.026486    0.014557     0.002123   0.002123   0.042762    0.002729   
98     0.011498    0.027900     0.002452   0.001691   0.045570    0.004396   
99     0.011448    0.016430     0.003180   0.001590   0.045898    0.004876   

    TRBV4-3*01  TRBV5-8*01  TRBV7-5*01  TRBV10-1*01  ...  TRBV7

## Sampling reads from the repertoire, not changing the gene usage distribution

In [13]:
sampled_repertoire = RepertoireSampling().sample(repertoire=dataset[0], old_usage_matrix=[norm_usage_table_v, norm_usage_table_j], new_usage_matrix=None, n=100000)

In [14]:
sampled_repertoire

Repertoire of 14942 clonotypes and 100000 cells:
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CASSFWTGQLNTEAFF cdr3nt_0
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CASSSPG_CQPQHF cdr3nt_1
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CASSPDSQSSGNTIYF cdr3nt_2
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CASSVVGAQSGANVLTF cdr3nt_3
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CASSYSESEAFF cdr3nt_4
sequencingDate                                                  2020 10 19
folder                                               2020/10_FMBA_NovaSeq6
R1                                  7701015208

## Resampling the repertoire using the new derived usages for V and J genes

In [15]:
dataset[0]

Repertoire of 23157 clonotypes and 7823059 cells:
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CASSFWTGQLNTEAFF cdr3nt_0
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CASSSPG_CQPQHF cdr3nt_1
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CASSPDSQSSGNTIYF cdr3nt_2
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CASSVVGAQSGANVLTF cdr3nt_3
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CASSYSESEAFF cdr3nt_4
sequencingDate                                                  2020 10 19
folder                                               2020/10_FMBA_NovaSeq6
R1                                  770101520

In [16]:
t0 = time.time()
resampled_repertoire = RepertoireSampling().sample(repertoire=dataset[0], old_usage_matrix=[norm_usage_table_v, norm_usage_table_j], new_usage_matrix=[z_score_usage_table_v, z_score_usage_table_j])
print(time.time() - t0)

4.897488832473755


In [17]:
resampled_repertoire

Repertoire of 22056 clonotypes and 7823059 cells:
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CASSFWTGQLNTEAFF cdr3nt_0
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CASSSPG_CQPQHF cdr3nt_1
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CASSVVGAQSGANVLTF cdr3nt_3
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CAISDPGGNSPLHF cdr3nt_8
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CASSYSKATRR_GNLSGANVLTF cdr3nt_5
sequencingDate                                                  2020 10 19
folder                                               2020/10_FMBA_NovaSeq6
R1                                  

## Resampling all the reperoires from a dataset

In [18]:
t0 = time.time()
resampled_dataset = dataset.resample(updated_segment_usage_tables=[z_score_usage_table_v, z_score_usage_table_j], threads=28)
print(time.time() - t0)


1361533
153.08800864219666


In [19]:
dataset[14]

Repertoire of 7433 clonotypes and 3957727 cells:
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CASRGGTGGPQETQYF cdr3nt_0
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CASSYRRPSSGNTIYF cdr3nt_1
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CASSLGGQGNGELFF cdr3nt_2
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CASSYSPGDNYEQYF cdr3nt_3
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CASSSANYGYTF cdr3nt_4
sequencingDate                                                  2020 12 18
folder                                           2020/12_FMBA_Novaseq8_DNA
R1                                   6900004301

In [20]:
resampled_dataset[14]

Repertoire of 7006 clonotypes and 3957727 cells:
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CASSYPGQ_CTGELFF cdr3nt_258
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CATRGNTEAFF cdr3nt_5
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CASRGGTGGPQETQYF cdr3nt_0
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CASSYCAGWGTDTQYF cdr3nt_368
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CASSSGTGSGNTIYF cdr3nt_779
sequencingDate                                                  2020 12 18
folder                                           2020/12_FMBA_Novaseq8_DNA
R1                                   6900

In [33]:
trbv_matrix = NormalizedSegmentUsageTable.load_from_repertoire_dataset(
    resampled_dataset, gene='TRB', segment_type='V').segment_usage_matrix

In [34]:
trbj_matrix = NormalizedSegmentUsageTable.load_from_repertoire_dataset(
    resampled_dataset, gene='TRB', segment_type='J').segment_usage_matrix

In [35]:
trbj_matrix

,TRBJ2-1*01,TRBJ1-3*01,TRBJ2-6*01,TRBJ1-5*01,TRBJ1-6*01,TRBJ1-1*01,TRBJ2-5*01,TRBJ2-4*01,TRBJ1-4*01,TRBJ2-3*01,TRBJ1-2*01,TRBJ2-7*01,TRBJ2-2P*01,TRBJ2-2*01
0,0.116748,0.043299,0.018045,0.090542,0.054407,0.148848,0.083333,0.013919,0.050644,0.100290,0.117746,0.095394,0.000000,0.066785
1,0.180040,0.034944,0.021858,0.071757,0.037532,0.111447,0.099511,0.018407,0.039402,0.126833,0.091746,0.088295,0.000000,0.078228
2,0.152109,0.031283,0.019707,0.071080,0.044294,0.115661,0.097675,0.015307,0.041519,0.110016,0.111260,0.121879,0.000000,0.068210
3,0.141896,0.034816,0.023991,0.076653,0.043300,0.113517,0.091867,0.023113,0.039789,0.129901,0.114687,0.093329,0.000000,0.073142
4,0.153048,0.037046,0.019877,0.074229,0.042187,0.129682,0.089974,0.014781,0.044069,0.110540,0.115360,0.096768,0.000046,0.072393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.153537,0.036428,0.016931,0.085027,0.048309,0.126671,0.099681,0.019125,0.044294,0.116405,0.095624,0.084903,0.000000,0.073064
96,0.174786,0.027598,0.021953,0.073594,0.037215,0.111645,0.116454,0.020698,0.035961,0.134644,0.096592,0.059795,0.000000,0.089065
97,0.097207,0.047064,0.022652,0.092039,0.049263,0.195514,0.017924,0.001539,0.052892,0.032989,0.175830,0.183528,0.000000,0.031559
98,0.175528,0.032560,0.024073,0.080966,0.033512,0.111188,0.090059,0.018012,0.040873,0.122012,0.088067,0.105559,0.000087,0.077503
